In [4]:
import pandas as pd
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [5]:
sp = pd.read_csv("All-seasons.csv")
sp.replace('\n',' ',regex = True,inplace=True)
sp.to_csv('sp.csv',index= False)
sp.head()
cartman = sp[sp.Character=="Cartman"]
stan = sp[sp.Character == "Stan"]

In [6]:
import torchtext

text_field = torchtext.data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True)       # to turn each character into an integer index
i = 0
for line in open('sp3.txt'):
        #print(line)
        i = i+1
        if i>=10:
            break
fields = [('line', text_field)]
dataset = torchtext.data.TabularDataset("sp3.txt", # name of the file
                                        "csv",  # fields are separated by a tab
                                        fields)
split = [0.6, 0.2,0.2]
train, valid, test =  dataset.split(split, stratified=False, strata_field=None, random_state=None)
text_field.build_vocab(train)
# print(text_field.vocab.stoi)
# print(text_field.vocab.itos)
train_iter = torchtext.data.BucketIterator(train,
                                           batch_size=32,
                                           sort_key=lambda x: len(x.line), # to minimize padding
                                           sort_within_batch=True,        # sort within each batch
                                           repeat=False)                   # repeat the iterator for multiple epochs
for i, batch in enumerate(train_iter):
    if i >= 10:
        break
    lines = batch.line[0]
    print(lines)

tensor([[39, 15,  5,  5,  3,  8, 11, 13, 38,  9,  2, 31, 26, 17,  2, 11,  6,  8,
          8, 14],
        [37, 14, 12,  3, 13, 41,  6, 15,  2, 29,  4, 11,  5,  4,  8, 16, 11, 22,
          2,  2],
        [46, 10,  8,  4,  5,  3, 11, 13, 41,  3,  3,  2,  9,  3,  3,  2,  9,  3,
          3,  2],
        [27,  4,  5,  4,  7, 13, 31,  2, 12,  6, 32,  3,  2, 14,  6, 15,  2,  5,
          6,  6],
        [34, 10, 20,  9,  4,  3, 12,  2, 28,  9,  3,  8,  5,  6, 25, 25, 13, 41,
          3, 11],
        [27,  5,  4,  7, 13, 30,  9,  4,  5,  2, 10, 11,  2,  5,  9,  4,  5, 35,
         22,  2],
        [42, 10,  8, 12, 13, 30,  9,  6,  2,  4,  8,  3,  2, 14,  6, 15, 35, 22,
          2,  2],
        [53,  7, 20, 12,  3,  2, 40,  6, 21,  4,  8, 16, 13, 38,  9, 18,  2, 40,
          3, 14],
        [28,  4,  8,  5, 17,  4,  7, 13, 33,  9,  3,  2,  5,  9, 10,  7, 19,  2,
         10, 11],
        [27,  5,  4,  7, 13, 31,  5,  2, 21,  4, 11,  2, 11,  6,  2, 19,  8,  6,
         11, 11],
        [3

          14,  26,  16,   2,  14,   6,  15,   2,  63,  15,  10,   5,  35,   2]])
tensor([[44,  4,  7,  ...,  5, 35,  2],
        [50, 17, 24,  ...,  2,  1,  1],
        [33,  6, 21,  ...,  1,  1,  1],
        ...,
        [27, 10,  7,  ...,  1,  1,  1],
        [36, 12,  2,  ...,  1,  1,  1],
        [28,  4,  8,  ...,  1,  1,  1]])
tensor([[37,  3,  7,  ..., 18, 61,  2],
        [27,  5,  4,  ..., 22,  2,  2],
        [27,  5,  4,  ...,  7,  3,  8],
        ...,
        [44, 10, 20,  ..., 85,  2,  1],
        [34,  4,  7,  ...,  2,  2,  1],
        [34,  4,  8,  ..., 18,  2,  1]])
tensor([[34,  8, 18,  ...,  4, 11, 11],
        [43,  4, 32,  ..., 21, 22,  2],
        [37, 14, 12,  ..., 23, 35,  2],
        ...,
        [34, 10, 17,  ..., 17, 14,  1],
        [39, 15,  5,  ..., 18,  2,  1],
        [44,  4,  7,  ..., 22,  2,  1]])
tensor([[37, 14, 12,  3, 13, 30,  9, 14],
        [28,  9,  3, 25, 13, 40,  3, 14],
        [44,  4,  7, 16, 14, 13, 53,  9],
        [37, 14, 12,  3, 13, 30

In [7]:
cartman_text = ""
for line in cartman.Line:
    cartman_text += line
# show the first 100 characters
stan_text = ''
for l in stan.Line:
    stan_text += l

In [8]:
vocab = list(set(cartman_text))
vocab_stoi = {s: i for i, s in enumerate(vocab)}
vocab_itos = {i: s for i, s in enumerate(vocab)}
len(vocab)

101

In [9]:
import random
random.seed(10)

cartman_len = len(cartman_text)

def random_chunk(chunk_len=300):
    """Return a random subsequence from `spam_text`"""
    start_index = random.randint(0, cartman_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return cartman_text[start_index:end_index]

print(random_chunk())

 All I have to do is be thinking about American history when I walk through this doorway. I'll trip the rope, causing the rock to fall on my head. And then I will have a flashback to the times 


In [10]:
def text_to_tensor(text, vocab=vocab):
    """Return a tensor containing the indices of characters in `text`."""
    indices = [vocab_stoi[ch] for ch in text]
    return torch.tensor(indices)

print(text_to_tensor(random_chunk()))

tensor([52, 31, 71, 42, 29,  9, 26, 71, 42,  7, 97,  7, 52,  9, 57, 42,  6, 58,
        71, 42, 54,  7, 36, 97, 42, 58,  7, 27, 42, 32, 40, 71, 98,  7, 52, 85,
        71, 22, 79, 42, 16, 42, 58, 29, 32, 71, 42, 49, 29,  1, 67, 98, 71, 42,
        85, 71,  7, 40,  9, 71, 22, 57, 42, 31, 52, 22, 27, 57, 42,  6, 58,  7,
         6, 42, 52, 28, 42, 49, 29,  1, 42, 22, 29,  9, 67,  6, 42, 97, 29, 42,
        36, 52,  6, 58, 42, 59, 58, 40, 52, 27,  6, 57, 42, 49, 29,  1, 42, 26,
        29,  1, 85, 22, 42, 71,  9, 22, 42,  1, 32, 42, 62,  1, 27,  6, 42, 85,
        52, 31, 71, 42,  6, 58,  7,  6, 42, 27, 32, 85,  7,  6,  6, 71, 40, 71,
        22, 42, 25, 52,  6, 26, 58, 42, 22, 29, 36,  9, 42, 29,  9, 42,  6, 58,
        71, 42, 32,  7, 98, 71, 55, 71,  9,  6, 79, 19, 42, 84, 58,  7,  6, 67,
        27, 42, 26, 29, 29, 85, 79, 42, 16, 67, 55, 42, 22, 29,  9, 71, 42, 55,
         7, 31, 52,  9, 97, 42, 55, 49, 42, 98, 52, 22, 71, 29, 42,  7,  9, 49,
        36,  7, 49, 27, 79, 42, 19, 42, 

In [11]:
def random_training_set(chunk_len=300):    
    chunk = random_chunk(chunk_len)
    inp = text_to_tensor(chunk[:-1])   # omit the last token
    target = text_to_tensor(chunk[1:]) # omit the first token
    return inp, target

In [48]:
##a naive model following the lecture notes with 1 hidden layer - GRU 
class CartmanLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers):
        
        super(CartmanLSTM, self).__init__()
        
        self.vocab_size = vocab_size
    
        self.hidden_size = hidden_size
        
        self.ident = torch.eye(vocab_size)
        
        self.rnn = nn.LSTM(vocab_size, hidden_size, n_layers)
        
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden):
        
        #set intiial hidden state and cell state
        h0 = torch.zeros(1, inp.size(0), self.hidden_size)
        c0 = torch.zeros(1, inp.size(0), self.hidden_size)
        
        #forward propogate
        out, _ = self.rnn(inp, (h0, c0))
        
        #passs output of last time step to classifier
        out = self.fc(out[:, -1, :])
        
        return out
        
    def init_hidden(self):
        return torch.zeros(self.vocab_size, 1, self.hidden_size)

In [49]:
def evaluate(model, prime_str='I', predict_len=300, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = text_to_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        predicted += predicted_char
        inp = text_to_tensor(predicted_char)

    return predicted

def train(model, num_iters=2000, lr=0.004):
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for it in range(num_iters):
        # get training set
        inp, target = random_training_set()
        # cleanup
        optimizer.zero_grad()
        # forward pass
        hidden = model.init_hidden()
        output, _ = model(inp, hidden)
        loss = criterion(output, target)
        # backward pass
        loss.backward()
        optimizer.step()

        if it % 200 == 199:
            print("[Iter %d] Loss %f" % (it+1, float(loss)))
            print("    " + evaluate(model, ' ', 300))


In [50]:
model = CartmanLSTM(len(vocab), 128)

TypeError: __init__() missing 1 required positional argument: 'n_layers'

In [51]:
train(model, num_iters=5000)

RuntimeError: input must have 3 dimensions, got 1